In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset

# Load the IMDb dataset for training and testing
train_data = load_dataset("imdb", split="train")
train_data = train_data.shard(num_shards=4, index=0)  # Use a subset of the data for faster processing

test_data = load_dataset("imdb", split="test")
test_data = test_data.shard(num_shards=4, index=0)  # Use a subset of the test data

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

* This code demonstrates how to load the IMDb dataset using Hugging Face's datasets library.
* The shard method is used to split the dataset into smaller parts to make training and testing more efficient.

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load a pre-trained model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]


* AutoModelForSequenceClassification loads a pre-trained model optimized for sequence classification tasks like sentiment analysis.
* AutoTokenizer prepares text data to be input into the model

In [ ]:
# Tokenize the training and testing datasets
tokenized_training_data = tokenizer(
    train_data["text"],
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=64
)

tokenized_test_data = tokenizer(
    test_data["text"],
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=64
)


* The tokenizer converts raw text into a format that the model can process (e.g., token IDs).
* The padding and truncation parameters ensure consistent input lengths, and max_length limits the input size to 64 tokens.

In [ ]:
# Define a function to tokenize individual rows
def tokenize_function(text_data):
    return tokenizer(
        text_data["text"],
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=64
    )

# Tokenize the dataset in batches
tokenized_in_batches = train_data.map(tokenize_function, batched=True)

# Tokenize the dataset row by row
tokenized_by_row = train_data.map(tokenize_function, batched=False)


Map:   0%|          | 0/6250 [00:00<?, ? examples/s]

Map:   0%|          | 0/6250 [00:00<?, ? examples/s]

* Tokenization can be performed either in batches (more efficient) or row by row.
* The map method applies the tokenize_function to each row or batch of rows in the dataset.

In [ ]:
from transformers import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="./finetuned",         # Where to save the fine-tuned model
    evaluation_strategy="epoch",     # Evaluate at the end of each epoch
    num_train_epochs=3,              # Number of training epochs
    learning_rate=2e-5,              # Learning rate
    per_device_train_batch_size=8,   # Batch size for training
    per_device_eval_batch_size=8,    # Batch size for evaluation
    weight_decay=0.01                # Weight decay to prevent overfitting
)

# Initialize the Trainer
trainer = Trainer(
    model=model,                     # The pre-trained model to fine-tune
    args=training_args,              # Training arguments
    train_dataset=tokenized_training_data,  # Tokenized training data
    eval_dataset=tokenized_test_data,       # Tokenized test data
    tokenizer=tokenizer              # Tokenizer used for preprocessing
)

# Start fine-tuning
trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-7-051cde6d3f40>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:


Abort: 

* TrainingArguments allows customization of the training process, such as learning rate, number of epochs, and batch size.
* Trainer simplifies the fine-tuning process by handling training, evaluation, and logging.

In [ ]:
# Example trainer output
{'eval_loss': 0.398524671792984, 'eval_runtime': 33.3145, 'eval_samples_per_second': 46.916,
 'eval_steps_per_second': 5.883, 'epoch': 1.0}

{'eval_loss': 0.1745782047510147, 'eval_runtime': 33.5202, 'eval_samples_per_second': 46.629,
 'eval_steps_per_second': 5.847, 'epoch': 2.0}

{'loss': 0.4272, 'grad_norm': 15.558795928955078, 'learning_rate': 2.993197278911565e-06,
 'epoch': 2.5510204081632653}

{'eval_loss': 0.12216147780418396, 'eval_runtime': 33.2238, 'eval_samples_per_second': 47.045,
 'eval_steps_per_second': 5.899, 'epoch': 3.0}


{'eval_loss': 0.12216147780418396,
 'eval_runtime': 33.2238,
 'eval_samples_per_second': 47.045,
 'eval_steps_per_second': 5.899,
 'epoch': 3.0}

These logs display the loss, runtime, evaluation speed, and progress for each epoch.

In [ ]:
pip install torch


In [ ]:
import torch

# Define new input data
new_data = ["This movie was disappointing!", "This is the best movie ever!"]

# Tokenize the input data
new_input = tokenizer(
    new_data,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=64
)

# Use the fine-tuned model to make predictions
with torch.no_grad():
    outputs = model(**new_input)

# Convert model predictions to labels
predicted_labels = torch.argmax(outputs.logits, dim=1).tolist()

# Map labels to sentiments
label_map = {0: "NEGATIVE", 1: "POSITIVE"}
for i, predicted_label in enumerate(predicted_labels):
    sentiment = label_map[predicted_label]
    print(f"Input Text {i + 1}: {new_data[i]}")
    print(f"Predicted Label: {sentiment}")


Input Text 1: This movie was disappointing!
Predicted Label: POSITIVE
Input Text 2: This is the best movie ever!
Predicted Label: POSITIVE


* This code predicts sentiments (positive or negative) for new input text using the fine-tuned model.
* It maps the predicted numerical labels to human-readable labels ("POSITIVE" or "NEGATIVE").

**Saving and Reloading Models**

In [ ]:
# Save the fine-tuned model and tokenizer
model.save_pretrained("my_finetuned_files")
tokenizer.save_pretrained("my_finetuned_files")

# Reload the saved model and tokenizer
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("my_finetuned_files")
tokenizer = AutoTokenizer.from_pretrained("my_finetuned_files")


* This code saves the fine-tuned model and tokenizer for later use.
* The saved files can be reloaded anytime to make predictions without re-training.